In [2]:
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
import os
from IPython.display import clear_output
from ipywidgets import widgets, interact

%matplotlib inline
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

class Poke_info:
#     pokemon = pd.read_csv(r"C:\Users\dudas\Documents\python\data_analysis\pokemon_data_analysis\pokemon_data.csv")
    pokemon = pd.read_csv(r"./pokemon_data.csv")
    total_stats = lambda row: (row.HP + row.Att + row.Def + row["S.Att"] + row["S.Def"])
    pokemon['Total Stats'] = pokemon.apply(total_stats, axis = 1)
    pokemon_use = pokemon.copy()
    for i in ["Hidden_Ability", "Ability1", "Ability2", "Weight_kg", "Height_m", "Height_ft", "Cap_Rate", \
              "Egg_Steps", "Classification", "Weight_lbs"]:
        del pokemon_use[i]
    types_list = ["None", "fire", "water", "grass", "electric", "ice", "fighting", "poison", \
                 "ground", "flying", "psychic", "bug", "rock", "ghost", "dragon", "dark",\
                 "steel", "fairy", "normal"]
    super_effective_to = {"normal": ["fighting"], "flying": ["electric", "ice", "rock"], "rock": ["fighting", "grass", \
                            "ground", "steel", "water"], "fire": ["ground", "rock", "water"], "fighting": ["fairy", "flying", "psychic"], \
                            "dragon": ["dragon", "fairy", "ice"], "ice": ["fighting", "fire", "rock", "steel"], "ground": ["grass", "ice", "water"],\
                            "poison": ["ground", "psychic"], "steel": ["fighting", "fire", "ground"], "fairy": ["poison", "steel"], \
                            "psychic": ["bug", "dark", "ghost"], "water": ["electric", "grass"], "electric": ["ground"], "dark": ["bug", "fairy", "fighting"], \
                            "ghost": ["dark", "ghost"], "bug": ["fire", "flying", "rock"], "grass": ["bug", "fire", "flying", "ice", "poison"]}
    super_effective_against = {"normal": ["None"], "flying": ["bug", "fighting", "grass"], "rock": ["bug", "fire", "flying", "ice"], \
                              "fire": ["bug", "grass", "ice", "steel"], "fighting": ["dark", "ice", "normal", "rock", "steel"],\
                              "dragon": ["dragon"], "ice": ["dragon", "flying", "grass", "ground"], "ground": ["electric", "fire", "poison", \
                              "rock", "steel"], "poison": ["fairy", "grass"], "steel": ["fairy", "ice", "rock"], "fairy": ["fighting", "dark", "dragon"], \
                              "psychic": ["fighting", "poison"], "water": ["fire", "ground", "rock"], "electric": ["flying", "water"], \
                              "dark": ["ghost", "psychic"], "ghost": ["ghost", "psychic"], "bug": ["grass", "dark", "psychic"], \
                              "grass": ["ground", "rock", "water"]}
    pokemon_list = pokemon["Name"].tolist()
    pokemon_list.sort()
    pokemon_list.append("None")
    layout = widgets.Layout(width='auto', height='auto')
    
    def __init__(self):
        self.pokemon_name = None
        self.output_dropdown_variable = None
        self.dex_pokemon = None
        self.pokemon_list_dropdown = None
        self.types_dropdown = None
        self.pokedex_selector = None
        self.dex_or_mon = None
        self.current_pokemon_str = None
        

    def start_display(self):
        print("\nTop Total Stats:\n(HP, Att, Def, Sp.Att, Sp.Def)")
        best_tstats = Poke_info.pokemon_use.sort_values(by ='Total Stats', ascending = False).reset_index(drop = True)
        display(best_tstats.head())
        print("\nTop Special Attack:")
        best_spatt = Poke_info.pokemon_use.sort_values(by ='S.Att', ascending = False).reset_index(drop = True)
        display(best_spatt.head())
        print("\nTop Attack:")
        best_attack = Poke_info.pokemon_use.sort_values(by ='Att', ascending = False).reset_index(drop = True)
        display(best_attack.head())
        print("\nTop Defense:")
        best_defense = Poke_info.pokemon_use.sort_values(by ='Def', ascending = False).reset_index(drop = True)
        display(best_defense.head())
        print("\nTop Special Defense:")
        best_spdefense = Poke_info.pokemon_use.sort_values(by ='S.Def', ascending = False).reset_index(drop = True)
        display(best_spdefense.head())
        
        total_stat_x = Poke_info.pokemon_use["Total Stats"].to_list()
        spatt_x = Poke_info.pokemon_use["S.Att"].to_list()
        
        att_x = Poke_info.pokemon_use["Att"].to_list()
        hp_x = Poke_info.pokemon_use["HP"].to_list()
        
        def_x = Poke_info.pokemon_use["Def"].to_list()
        spdef_x = Poke_info.pokemon_use["S.Def"].to_list()
        
        f = plt.figure(figsize=(12, 5))
        ax = f.add_subplot(121)
        plt.title("Total Stats Spread")
        plt.xlabel('Total Stats Value')
        plt.ylabel('Number of Pokemon')
        ax2 = f.add_subplot(122)
        ax.hist(total_stat_x, histtype = "step")
        ax2.hist(spatt_x, histtype = "step")
        plt.title("Special Attack Spread")
        plt.xlabel('Special Attack Value')
        plt.ylabel('Number of Pokemon')
        plt.subplots_adjust(top=0.95, hspace=2.0)
        
        
        
        f2 = plt.figure(figsize=(12, 5))
        ax3 = f2.add_subplot(121)
        plt.title("Attack Spread")
        plt.xlabel('Attack Value')
        plt.ylabel('Number of Pokemon')
        ax4 = f2.add_subplot(122)
        ax3.hist(att_x, histtype = "step")
        ax4.hist(hp_x, histtype = "step")
        plt.title("HP Spread")
        plt.xlabel('HP Value')
        plt.ylabel('Number of Pokemon')
        plt.subplots_adjust(top=0.95, hspace=2.0)
        
        f3 = plt.figure(figsize=(12, 5))
        ax5 = f3.add_subplot(121)
        plt.title("Defense Spread")
        plt.xlabel('Defense Value')
        plt.ylabel('Number of Pokemon')
        ax6 = f3.add_subplot(122)
        ax5.hist(att_x, histtype = "step")
        ax6.hist(hp_x, histtype = "step")
        plt.title("Special Defense Spread")
        plt.xlabel('Special Defense Value')
        plt.ylabel('Number of Pokemon')
        plt.subplots_adjust(top=0.95, hspace=2.0)
        
        self.start_buttons()

    def info_by_type(self, Type):
        self.output_dropdown_variable = str(self.types_dropdown.value)
        current_type = str(self.output_dropdown_variable)
        type_stats = Poke_info.pokemon_use[(Poke_info.pokemon_use.Type1 == current_type)|(Poke_info.pokemon_use.Type2 \
                                                                                          == current_type)]
        type_stats.reset_index(drop=True)
        if current_type != "None":
            print("\n\nSuper effective Against: ")
            for item in Poke_info.super_effective_against.get(current_type):
                print(item)
            print("\nWeak to: ")
            for item in Poke_info.super_effective_to.get(current_type):
                print(item)
            print("\nTop Total Stats - {0} Type Pokemon:\n(HP, Att, Def, Sp.Att, Sp.Def)".format(current_type.title()))
            best_type_tstats = type_stats.sort_values(by ='Total Stats', ascending = False).reset_index(drop = True)
            display(best_type_tstats.head())
            print("\nTop Special Attack - {0} Type Pokemon:".format(current_type.title()))
            best_type_spatt = type_stats.sort_values(by ='S.Att', ascending = False).reset_index(drop = True)
            display(best_type_spatt.head())
            print("\nTop Normal Attack - {} Type Pokemon:".format(current_type.title()))
            best_type_attack = type_stats.sort_values(by ='Att', ascending = False).reset_index(drop = True)
            display(best_type_attack.head())
        self.back_to_start()
    
    def all_poke_chart_display(self, pokemon_stats):
        x_pos, y_pos = None, 12
        print("\n\n\n\n\n\n\nComparison Charts for All Pokemon")
        total_stat_x = Poke_info.pokemon_use["Total Stats"].to_list()
        spatt_x = Poke_info.pokemon_use["S.Att"].to_list()
        att_x = Poke_info.pokemon_use["Att"].to_list()
        hp_x = Poke_info.pokemon_use["HP"].to_list()
        
        f = plt.figure(figsize=(7, 4))
        ax = f.add_subplot(121)
        plt.title("Total Stats for {0} vs \nAll Pokemon".format(self.current_pokemon_str))
        plt.xlabel('Total Stats Value')
        plt.ylabel('Number of Pokemon')
        x_pos = pokemon_stats["Total Stats"].to_list()[0]
        ax.plot(x_pos,y_pos,'r*')
        ax2 = f.add_subplot(122)
        ax.hist(total_stat_x, histtype = "step")
        ax2.hist(spatt_x, histtype = "step")
        x_pos = pokemon_stats["S.Att"].to_list()[0]
        ax2.plot(x_pos,y_pos,'r*')
        plt.title("Sp. Attack for {0} vs \nAll Pokemon".format(self.current_pokemon_str))
        plt.xlabel('Sp. Attack Value')
        plt.ylabel('Number of Pokemon')
        plt.subplots_adjust(top=0.95, hspace=20.0)
        
        f2 = plt.figure(figsize=(7, 4))
        ax3 = f2.add_subplot(121)
        plt.title("Attack for {0} vs \nAll Pokemon".format(self.current_pokemon_str))
        plt.xlabel('Attack Value')
        plt.ylabel('Number of Pokemon')
        x_pos = pokemon_stats["Att"].to_list()[0]
        ax3.plot(x_pos,y_pos,'r*')
        ax4 = f2.add_subplot(122)
        ax3.hist(att_x, histtype = "step")
        ax4.hist(hp_x, histtype = "step")
        plt.title("HP for {0} vs \nAll Pokemon".format(self.current_pokemon_str))
        plt.xlabel('HP Value')
        plt.ylabel('Number of Pokemon')
        
        x_pos = pokemon_stats["HP"].to_list()[0]
        ax4.plot(x_pos,y_pos,'r*')
        plt.subplots_adjust(top=0.95, hspace=20.0)
    
    def comparison_charts_creator(self, indiv_stats_chart):
        x_pos, y_pos = None, 4
        current_types_list = []
        current_types_list.append(self.type1_str)
        if self.type2_str != None:
            current_types_list.append(self.type2_str)
            
        current_types_chart = Poke_info.pokemon_use[(Poke_info.pokemon_use.Type1.isin(current_types_list))|(Poke_info.pokemon_use.Type2.isin(current_types_list))].reset_index(drop = True)
        
        total_stat_x = current_types_chart["Total Stats"].to_list()
        spatt_x = current_types_chart["S.Att"].to_list()
        
        att_x = current_types_chart["Att"].to_list()
        hp_x = current_types_chart["HP"].to_list()
        
        f = plt.figure(figsize=(7, 4))
        ax = f.add_subplot(121)
        plt.title("Total Stats {0} vs \nSame Type(s)".format(self.current_pokemon_str))
        plt.xlabel('Total Stats')
        plt.ylabel('Number of Pokemon')
        ax2 = f.add_subplot(122)
        ax.hist(total_stat_x, histtype = "step")
        x_pos = indiv_stats_chart["Total Stats"].to_list()[0]
        ax.plot(x_pos,y_pos,'r*')
        ax2.hist(spatt_x, histtype = "step")
        x_pos = indiv_stats_chart["S.Att"].to_list()[0]
        ax2.plot(x_pos,y_pos,'r*')
        plt.title("Sp. Attack {0} vs \nSame Type(s)".format(self.current_pokemon_str))
        plt.xlabel('Sp. Attack Value')
        plt.ylabel('Number of Pokemon')
        plt.subplots_adjust(top=0.95, hspace=20.0)
        
        f2 = plt.figure(figsize=(7, 4))
        ax3 = f2.add_subplot(121)
        plt.title("Attack {0} vs \nSame Type(s)".format(self.current_pokemon_str))
        plt.xlabel('Attack Value')
        plt.ylabel('Number of Pokemon')
        x_pos = indiv_stats_chart["Att"].to_list()[0]
        ax3.plot(x_pos,y_pos,'r*')
        ax4 = f2.add_subplot(122)
        ax3.hist(att_x, histtype = "step")
        ax4.hist(hp_x, histtype = "step")
        x_pos = indiv_stats_chart["HP"].to_list()[0]
        ax4.plot(x_pos,y_pos,'r*')
        plt.title("HP {0} vs \nSame Type(s)".format(self.current_pokemon_str))
        plt.xlabel('HP Value')
        plt.ylabel('Number of Pokemon')
        plt.subplots_adjust(top=0.95, hspace=20.0)

    def display_poke_info(self, Pokemon):
        check_variable = "None"
        if self.dex_or_mon == "mon":
            self.pokemon_name = str(self.pokemon_list_dropdown.value)
            current_pokemon = str(self.pokemon_name)
            pokemon_stats = Poke_info.pokemon[Poke_info.pokemon.Name == current_pokemon].reset_index(drop = True)
            check_variable = current_pokemon
        elif self.dex_or_mon == "dex":
            self.dex_pokemon = Poke_info.pokemon_use[Poke_info.pokemon_use["No."] == Pokemon]["Name"].to_list()
            self.dex_pokemon = self.dex_pokemon[0]
            pokemon_stats = Poke_info.pokemon[Poke_info.pokemon.Name == self.dex_pokemon].reset_index(drop = True)
            check_variable = self.dex_pokemon
        self.current_pokemon_str = check_variable
        if check_variable != "None":
            display(pokemon_stats)
            self.type1_str = pokemon_stats.Type1.to_string(index = False)[1:]
            type2_null = pokemon_stats.Type2.isnull().to_string(index = False)[1:]
            self.type2_str = None if type2_null == "True" else pokemon_stats.Type2.to_string(index = False)[1:]
            se_string = "Super effective against:"
            print("\033[1m" + se_string + "\033[0m")
            if type2_null == "False":
                print("for Type 1: {}".format(self.type1_str))
            items = ""
            counter= 0
            for item in Poke_info.super_effective_against.get(self.type1_str):
                if counter == 0:
                    items = items + item
                else:
                    items = items + ", " + item
                counter += 1
            print(items)
            items, counter = "", 0
            if self.type2_str != None:
                print("\nfor Type 2: {}".format(self.type2_str))
                for item in Poke_info.super_effective_against.get(self.type2_str):
                    if counter == 0:
                        items = items + item
                    else:
                        items = items + ", " + item
                    counter += 1
            print(items)
            we_string = "\nWeak to:"
            print("\033[1m" + we_string + "\033[0m")
            items, counter = "", 0
            if type2_null == "False":
                print("for Type 1: {}".format(self.type1_str))
            for item in Poke_info.super_effective_to.get(self.type1_str):
                if counter == 0:
                    items = items + item
                else:
                    items = items + ", " + item
                counter += 1
            print(items)
            items, counter = "", 0
            if self.type2_str != None:
                print("\nfor Type 2: {}".format(self.type2_str))
                for item in Poke_info.super_effective_to.get(self.type2_str):
                    if counter == 0:
                        items = items + item
                    else:
                        items = items + ", " + item
                    counter += 1
                print(items)
            self.view_strongest_weak_to()
            self.view_strongest_strong_against()
            self.comparison_charts_creator(pokemon_stats)
            self.all_pokemon_comparison_charts(pokemon_stats)
        self.back_to_start()

    def stats_by_type_command(self, _):
        clear_output()
        self.types_dropdown = widgets.Dropdown(style = {'description_width': 'initial'},
        options=Poke_info.types_list,
        value='None',
        description="Pick the Type you would like to Search",
        disabled=False,)
        self.type_dropdown = interact(self.info_by_type, Type= self.types_dropdown)

    def stats_by_pokemon(self, _):
        self.dex_or_mon = "mon"
        clear_output()
        self.pokemon_list_dropdown = widgets.Dropdown(style ={'description_width': 'initial'},
        options= Poke_info.pokemon_list,
        value='None',
        description= "Select your Pokemon",
        disabled=False,)
        self.pokemon_dropdown = interact(self.display_poke_info, Pokemon = self.pokemon_list_dropdown)

    def stats_by_pokedex(self, _):
        self.dex_or_mon = "dex"
        clear_output()
        self.pokedex_selector = widgets.Dropdown(
        style ={'description_width': 'initial'},
        options= list(range(1,401)),
        value=1,
        description= "Select your Pokedex Number",
        disabled=False,)
        interact(self.display_poke_info, Pokemon = self.pokedex_selector)

    def delete_widgets(self):
        self.dex_or_mon, self.current_pokemon_str, self.type_1_str, self.type_2_str = None, None, None, None
        opts = [self.pokemon_name, self.output_dropdown_variable, self.dex_pokemon]
        opts2 = [self.pokemon_list_dropdown, self.types_dropdown, self.pokedex_selector]
        for i in opts:
            if i != None:
                x = opts.index(i)
                opts2[x].close()
            
    def strongest_against_chart(self):
        print("\n\n\nStrongest Pokemon {0} is Super Effective Against: ".format(self.current_pokemon_str))
        sea_list = []
        sea_list.extend(Poke_info.super_effective_against.get(self.type1_str))
        if self.type2_str != None:
            sea_list.extend(Poke_info.super_effective_against.get(self.type2_str))
            
        sea_chart = Poke_info.pokemon_use[(Poke_info.pokemon_use.Type1.isin(sea_list))|(Poke_info.pokemon_use.Type2.isin(sea_list))]
        sea_chart = sea_chart.sort_values(by ='Total Stats', ascending = False).reset_index(drop = True)
        display(sea_chart.head(10))
    
    def weakest_against_chart(self):
        print("\n\n\nStrongest Pokemon {0} is Weak To:".format(self.current_pokemon_str))
        wa_list = []
        wa_list.extend(Poke_info.super_effective_to.get(self.type1_str))
        if self.type2_str != None:
            wa_list.extend(Poke_info.super_effective_to.get(self.type2_str))
        wa_chart = Poke_info.pokemon_use[(Poke_info.pokemon_use.Type1.isin(wa_list))|(Poke_info.pokemon_use.Type2.isin(wa_list))]
        wa_chart = wa_chart.sort_values(by ='Total Stats', ascending = False).reset_index(drop = True)
        display(wa_chart.head(10))
        
    def start_buttons(self):
        print("\nMore Options")
        stats_by_type_but = widgets.Button(description = "Search Stats by Type")
        stats_by_type_but.on_click(lambda x: [clear_output(), self.stats_by_type_command(x)])
        display(stats_by_type_but)

        stats_by_pokemon_but = widgets.Button(description = "Search Data by Pokemon", layout = Poke_info.layout)
        stats_by_pokemon_but.on_click(lambda x: [clear_output(), self.stats_by_pokemon(x)])
        display(stats_by_pokemon_but)

        stats_by_pokedex_but = widgets.Button(description = "Search Data by Pokedex Number", layout = Poke_info.layout)
        stats_by_pokedex_but.on_click(lambda x: [clear_output(), self.stats_by_pokedex(x)])
        display(stats_by_pokedex_but)

    def back_to_start(self):
        back_to_start_but = widgets.Button(description = "Return to start page", layout = Poke_info.layout)
        back_to_start_but.on_click(lambda x: [clear_output(), self.delete_widgets(), self.start_display()])
        display(back_to_start_but)
    
    def view_strongest_weak_to(self):
        strong_to_but = widgets.Button(description = "View Strongest Pokemon {0} is Super Effective Against".format(self.current_pokemon_str), layout = Poke_info.layout)
        strong_to_but.on_click(lambda x: [self.strongest_against_chart()])
        display(strong_to_but)
        
    def view_strongest_strong_against(self):
        weak_to_but = widgets.Button(description = "View Strongest Pokemon {0} is Weak To".format(self.current_pokemon_str), layout = Poke_info.layout)
        weak_to_but.on_click(lambda x: [self.weakest_against_chart()])
        display(weak_to_but)
    
    def all_pokemon_comparison_charts(self, pokemon_stats):
        comp_all_poke_but = widgets.Button(description = "View Comparison Charts for {} vs All Pokemon".format(self.current_pokemon_str), layout = Poke_info.layout)
        comp_all_poke_but.on_click(lambda x: [self.all_poke_chart_display(pokemon_stats)])
        display(comp_all_poke_but)
        

search_1 = Poke_info()
search_1.start_display()







interactive(children=(Dropdown(description='Select your Pokemon', index=400, options=('Abomasnow', 'Accelgor',…